In [1]:
import os
import ray
import ray.util
from ray.util.client import ray as rayclient

In [2]:
headhost = os.environ['RAY_CLUSTER']

if not rayclient.is_connected():
    rx = ray.util.connect('{ray_head}:10001'.format(ray_head=headhost))
rx

{'num_clients': 1,
 'python_version': '3.8.6',
 'ray_version': '1.4.0',
 'ray_commit': '3a09c82fbfce8f00533234844729e6d99fb0f24c',
 'protocol_version': '2021-05-20'}

In [3]:
from xgboost_ray import RayDMatrix, RayParams, train
from sklearn.datasets import load_breast_cancer

train_x, train_y = load_breast_cancer(return_X_y=True)
train_set = RayDMatrix(train_x, train_y)

In [4]:

evals_result = {}
bst = train(
    {
        "objective": "binary:logistic",
        "eval_metric": ["logloss", "error"],
    },
    train_set,
    evals_result=evals_result,
    evals=[(train_set, "train")],
    verbose_eval=False,
    ray_params=RayParams(
        num_actors=2,  # Number of remote actors
        cpus_per_actor=1))

bst.save_model("model.xgb")


(pid=266) 2021-06-28 15:43:39,312	INFO main.py:853 -- [RayXGBoost] Created 2 new actors (2 total actors). Waiting until actors are ready for training.
(pid=266) 2021-06-28 15:44:09,372	INFO main.py:887 -- Waiting until actors are ready (30 seconds passed).
(pid=266) 2021-06-28 15:44:39,426	INFO main.py:887 -- Waiting until actors are ready (60 seconds passed).
(pid=266) 2021-06-28 15:44:40,732	INFO main.py:898 -- [RayXGBoost] Starting XGBoost training.
(pid=116, ip=10.128.2.219) [15:44:40] task [xgboost.ray]:140064390102608 got new rank 0
(pid=343) [15:44:40] task [xgboost.ray]:140072127655648 got new rank 1
(pid=266) 2021-06-28 15:44:43,055	INFO main.py:1355 -- [RayXGBoost] Finished XGBoost training on training data with total N=569 in 163.80 seconds (2.31 pure XGBoost training time).


In [5]:
print("Final training error: {:.4f}".format(
    evals_result["train"]["error"][-1]))

Final training error: 0.0053
